In [72]:
#imports for machine learning and parsing
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split # To split our data for fair testing
from sklearn.linear_model import LogisticRegression  # The model itself!
from sklearn import preprocessing #LabelEncoders
import math #for ceil
import json

#server imports
from flask import Flask
from flask import request
from flask import jsonify

In [73]:
#loading our dataframe
diabetes_dataframe = pd.read_csv("./datasets/diabetes_prediction_dataset.csv")



In [74]:
#replacements
#To convert categorical data into numerical data, I googled it and you use labelencoding.
genderEncoder = preprocessing.LabelEncoder()
smokingEncoder = preprocessing.LabelEncoder()

diabetes_dataframe["gender"] = genderEncoder.fit_transform(diabetes_dataframe["gender"])
diabetes_dataframe["smoking_history"] = smokingEncoder.fit_transform(diabetes_dataframe["smoking_history"])


In [75]:

#splitting
features = diabetes_dataframe[['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level']]
outputs = diabetes_dataframe[['diabetes']].values.ravel()

features_train, features_test, outputs_train, outputs_test = train_test_split(features, outputs, test_size=0.1, random_state=4589)

In [76]:
#model preperation and training
model = LogisticRegression(max_iter=1000)
model.fit(features_train, outputs_train)

prediction_test = model.predict(features_test)
prediction_train = model.predict(features_train)

In [77]:
def test():
  error = 0
  for i in range(0, len(outputs_test)):
    error+=(float(prediction_test[i])-float(outputs_test[i]))**2
  error/=len(outputs_test)
  error**=0.5
  print(f"Error: {error}")
  print(f"Score: {model.score(features_test,outputs_test)}")


In [78]:
class Predictor:

    dataframe: pd.DataFrame

    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe

    def predict(self):
        return model.predict(self.dataframe)
    
    @staticmethod
    def parse_json(unparsed_json: str):
        parsed_json = json.loads(unparsed_json)
        return Predictor.json_to_frame(parsed_json)
    
    @staticmethod
    def json_to_frame(parsed_json: dict):
        return Predictor(pd.DataFrame([parsed_json]))

    def convert_binary_values(self):
        self.dataframe["gender"] = genderEncoder.transform(self.dataframe["gender"])
        self.dataframe["smoking_history"] = smokingEncoder.transform(self.dataframe["smoking_history"])

        return self
    


jsonPredict = Predictor.parse_json("""{
    \"gender\" : \"Male\",
    \"age\" : 12.0,
    \"hypertension\": 1,
    \"heart_disease\": 1,
    \"smoking_history\": \"current\",
    \"bmi\" : 23.86,
    \"HbA1c_level\": 4.8,
    \"blood_glucose_level\": 157.0
}""").convert_binary_values().predict()

print(type(jsonPredict[0]))


<class 'numpy.int64'>


In [79]:
#Serverside Setup

app = Flask(__name__)



In [80]:
#App Route
@app.route("/generate", methods=["GET"])
def generate():
    clientJson = request.json
    res =  float(Predictor.json_to_frame(clientJson).convert_binary_values().predict().tolist()[0])
    print(res)

    return jsonify({"result": res})

In [ ]:
#Run

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2025 07:06:10] "GET /generate HTTP/1.1" 200 -


0.0
